In [1]:
import twitter
from tweepy import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json

In [2]:
import yaml
import os
os.chdir('../')

In [3]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
twitter_cfg = cfg['TWEETER']

In [4]:
access_token = twitter_cfg['access_token']
access_token_secret = twitter_cfg['access_token_secret']
consumer_key = twitter_cfg['consumer_key']
consumer_secret = twitter_cfg['consumer_secret']

In [5]:
queue_cfg = cfg['QUEUES']
tweets_folder = queue_cfg['new_tweets']

In [6]:
tweets_folder

'../shared_folder/tweets'

In [7]:
#for folder in [tweets_folder, sentiment_folder, geo_folder, 'tweet_no_coordinates']:
for folder in [tweets_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [8]:
def save_tweet(tweet_id, json_data):
    print(tweet_id)
    filename = '{}/{}.json'.format(tweets_folder, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(json_data)
        fp.close()

In [ ]:
#Import the necessary methods from tweepy library
#from tweepy.streaming import StreamListener

#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        s = str(data)
        json_data = json.loads(s)
        if not 'id_str' in json_data:
            print('Invalid tweet:\n{}'.format(data))
            return False

        tweet_id = json_data['id_str']
        coordinates = json_data['coordinates']
        text = json_data['text']
                
        save_tweet(tweet_id, data)        
        return True

    def on_error(self, status):
        print (status)

In [ ]:
#This handles Twitter authetification and the connection to Twitter Streaming API
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)
stream.filter(locations=[111,-44,157,-10], languages=['en'])

990914495257001984
990914504128016388
990914517679783936
990914520611667968
990914525577658369
990914538563289088
990914543592161282
990914555856371713
990914567403327488
990914570687537154
990914588328714242
990914589998043136
990914593030524929
990914601477902336
990914603453374464
990914619043598336
990914617873346560
990914641080430592
990914643500580864
990914648357523457
990914680498487296
990914685489831936
990914694134120448
990914699729383425
990914722189819904
990914724165398528
990914737109176320
990914738207903744
990914752581812225
990914775797264384
990914776371875840
990914778255187969
990914793472077825
990914825801711616
990914830604292096
990914837084450816
990914836031619072
990914851533803520
990914857510756352
990914862694858752
990914864603250688
990914865211428864
990914866473975808
990914872027373568
990914873071755264
990914881200144384
990914895410487296
990914895595028480
990914895934775297
990914900879814658
990914908781821952
990914926624456704
990914939983

990918159371833344
990918194352144384
990918211729162240
990918235515109376
990918237993893888
990918259934351362
990918265252728832
990918267521847296
990918276497555457
990918281476259840
990918287600029697
990918289604820992
990918294893879296
990918294885482496
990918295300734977
990918302787489793
990918303433543682
990918313600475136
990918344189472768
990918353194704898
990918366813609984
990918373482553346
990918387839586305
990918393946505216
990918401848631296
990918404964937730
990918411780685825
990918423734403078
990918437592420352
990918437797937153
990918440775921665
990918448682188800
990918457490198533
990918483373309953
990918496664997888
990918499705880576
990918500339236864
990918539564408833
990918543754571776
990918544341647361
990918552449310723
990918555934715904
990918564759547904
990918566252695552
990918567011934208
990918574133755905
990918575534759937
990918581247356928
990918583461990400
990918587526266880
990918595176628225
990918609705693184
990918612104

990921460297289729
990921461056520192
990921474742538240
990921476311171072
990921478131535872
990921499652452352
990921504467668992
990921513556627457
990921538365935616
990921541071220736
990921560264355840
990921582846488576
990921588240351232
990921596758978562
990921605948751872
990921608054226946
990921613112557568
990921615826345984
990921618800209920
990921633828192257
990921645685596160
990921661300981760
990921697837568002
990921701062950913
990921714887413760
990921719828303873
990921722076385280
990921737368879104
990921746520817664
990921748546662400
990921749217796096
990921752447340545
990921760311672833
990921762337538048
990921762677260289
990921769719554048
990921771934101505
990921784546344961
990921798874054656
990921799557775360
990921809036890112
990921811410935809
990921826703327233
990921829211500545
990921831300263938
990921838032183297
990921853664354304
990921864108130304
990921883720691714
990921893724078086
990921918327832576
990921927698006016
990921927886

990925113057935360
990925115599699969
990925119072555013
990925119722733568
990925122201669632
990925130552373248
990925132565594112
990925188182130688
990925187968217088
990925202715435008
990925245023318016
990925246927589376
990925261687308289
990925268045856769
990925268901597184
990925276207968257
990925279190122499
990925284567207936
990925287918456832
990925289675931649
990925304150409216
990925301587701760
990925306503417856
990925316448075776
990925332998860801
990925340418584576
990925354704355330
990925372211384320
990925383745892352
990925396118986752
990925409851097088
990925415236616193
990925417006563334
990925421792321536
990925426741555200
990925434215788544
990925453358653441
990925500796190720
990925508450762752
990925519355920384
990925533360816129
990925546304389121
990925552449028097
990925554009362432
990925558329495552
990925584619323398
990925587400179713
990925615313408000
990925631553761280
990925634951098369
990925635726950400
990925636649697281
990925650646

990929393148051456
990929398063775745
990929406414802950
990929427981725696
990929444326981632
990929449351720961
990929449607618567
990929449985032192
990929447535575042
990929448634433536
990929487301820416
990929487947710464
990929490149687297
990929505530212357
990929520390754304
990929520822706177
990929543513849857
990929562644004864
990929562748923904
990929567484264448
990929568277057536
990929581963075584
990929583456186369
990929608961765377
990929614527610880
990929620911382528
990929625025925120
990929635595636736
990929648606265344
990929655975759873
990929666260152320
990929675856756736
990929680294330368
990929700741509120
990929739975049216
990929751199113217
990929754357252097
990929754931920898
990929756169252864
990929757725339648
990929769767174145
990929788238888961
990929788784132096
990929792672219136
990929808153432069
990929824670670848
990929825182334976
990929826113556482
990929849719054339
990929868324913152
990929880895307777
990929913120083968
990929923228

...

991095440694116352
991095445219753984
991095447220510720
991095466262609920
991095490660913152
991095499406036992
991095506813177858
991095507198984192
991095513180094464
991095513142317058
991095515449192448
991095529042952192
991095530452271104
991095539746816000
991095540078100480
991095541181308929
991095541785214976
991095554212941824
991095565340438528
991095570659004416
991095584483295232
991095590544031745
991095593794584577
991095595216527360
991095599108952066
991095626640248832
991095628062081024
991095629261619200
991095630385721345
991095632310894592
991095631635779584
991095635179855872
991095642024919040
991095662644150273
991095742591918080
991095744235896832
991095751190241286
991095756600705024
991095777966542848
991095784622899200
991095793217036289
991095793745518592
991095796249477126
991095800036966400
991095800716390400
991095816398913536
991095831066431488
991095840625192960
991095850863546368
991095857993809920
991095858539053056
991095870199291904
991095878453

991100143607803904
991100171915153408
991100178286370817
991100188805685248
991100189963206656
991100193733922816
991100195856310274
991100202168680448
991100205784338432
991100207499591681
991100252630351872
991100257009258497
991100271240429568
991100291679338496
991100292992188417
991100296054034432
991100305331777537
991100309576474624
991100309706457088
991100309953900544
991100321618317312
991100323975458816
991100330887659520
991100337187577856
991100351020347392
991100353952104449
991100361510346752
991100361460023296
991100402648039424
991100411019841536
991100419219771393
991100423921483776
991100441302786049
991100442883993600
991100444263907328
991100466070212611
991100466514685952
991100470176317441
991100471166222337
991100491726704640
991100511687340033
991100514204094465
991100519346200577
991100557094830080
991100566649520128
991100600245895168
991100623146946562
991100634609864704
991100643535273984
991100669934297088
991100669661593601
991100689454546944
991100692579

991104567050878976
991104569730990080
991104569873584128
991104570808938497
991104577826013184
991104584549416961
991104597153398784
991104603566452736
991104621048352769
991104625628475392
991104625280405504
991104629944430592
991104646075813890
991104656183971840
991104656725106688
991104702837211136
991104702782861312
991104703671996416
991104707488768000
991104758374006784
991104759888134144
991104773737795584
991104777718087680
991104780192788481
991104781912453120
991104795460059136
991104797402005504
991104825382256640
991104832810270720
991104835746385920
991104850304712704
991104887856365568
991104896597352448
991104897159389185
991104920081256448
991104921050099712
991104927219924992
991104931921739776
991104952972951562
991104960640172033
991104980042989568
991105065174818816
991105075048312833
991105090655211520
991105100205576192
991105123240759297
991105124276682752
991105150826639360
991105161614409729
991105165926125570
991105175644327936
991105206287912960
991105234691

991109200544608256
991109217737031680
991109231196569600
991109231641317376
991109231628533760
991109235143405568
991109249349517312
991109251517972480
991109278491557888
991109303137329152
991109325601980416
991109343595540481
991109356526563328
991109364273463297
991109367259779072
991109372032925696
991109373542858752
991109377703620608
991109402039013376
991109402307407872
991109416299642880
991109427876024321
991109455398895616
991109464542429184
991109466543173632
991109472226394112
991109497039998977
991109506863001601
991109527226302464
991109527813611525
991109533631102976
991109537519161344
991109569345564672
991109570058596353
991109614824448000
991109627671560192
991109637926567936
991109649595170816
991109650572562439
991109654800285696
991109665294450688
991109683757789184
991109694851768320
991109700706910208
991109733808357376
991109757355339776
991109758944882688
991109766872162304
991109783410262016
991109785469648896
991109790960005120
991109790981017600
991109807879

991122995975352320
991122997384765440
991123008633761792
991123010785497088
991123046797737984
991123056146792449
991123065256919040
991123070214537217
991123079496581120
991123098417025025
991123117450838016
991123121322184704
991123122270060545
991123130360905728
991123152775335938
991123171217571840
991123172589125632
991123180906430464
991123181686550528
991123181736898560
991123181699186689
991123229094830085
991123232483786753
991123246652248064
991123250678611968
991123259977428993
991123277744492544
991123295205388288
991123319708573697
991123323240173568
991123323823288320
991123355926282240
991123358249975808
991123364658868224
991123376558112768
991123396422320128
991123405834338304
991123414894039040
991123421982437377
991123440420634624
991123446879862784
991123456941932544
991123461027151872
991123475447230465
991123512139042816
991123532070318080
991123547329150976
991123546326761472
991123581122707457
991123593835659264
991123608855441410
991123613519523841
991123614844